<a href="https://colab.research.google.com/github/giuliocerruto/MC-uncertainty-embedding/blob/main/Homework3_RM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework III - Metodi quantitativi per la gestione del rischio
## Simona Maria Borrello s277789 
## Giulio Cerruto s277335 <a class="tocSkip">

In [ ]:
import numpy as np
from numpy.random import default_rng
np.random.seed(seed=2021) # settiamo il random seed
import pandas as pd
import plotly.graph_objects as go
!pip install quadprog
import quadprog
from plotly.subplots import make_subplots

Obiettivo di questo homework è verificare l’impatto degli **errori di stima** sulla formazione di un **portafoglio MV**.

L'analisi si focalizzerà sui seguenti punti: 
- confronto portafoglio di **2 asset** vs  portafoglio di **10 asset**; 
- effetto dell'**orizzonte temporale** su cui si basano le stime ( 7 giorni, 1 mese, 4 mesi) 
- effetto dell'**avversione** al **rischio** sulla stima del  portafoglio MV ottimo; 
- studio sugli  errori di stima del portafoglio a **minima varianza**;
- influenza del vincolo di **vendita** allo **scoperto** nel problema di ottimizzazione; 
- confronto tra il **peso degli errori** di stima dei rendimenti attesi vs errori di stima della matrice di varianza-covarianza dei rendimenti; 
- stima della **ricchezza terminale** su orizzenti temporali diversi e calcolo di relative **misure di rischio**. 







## Generazione dei dati
Per le analisi da svolgere utilizziamo **due portafogli** di numerosità differente: uno formato da **due soli asset** e uno con **dieci asset**. L'idea è quella di usare il portafoglio con due asset per comprendere più semplicemente gli effetti di certe manipolazioni e poi estendere il ragionamento al portafoglio con più asset. \
Iniziamo generando i **rendimenti del portafoglio** con due asset. Per ottenerli, campioneremo i dati da delle **distribuzioni normali multivariate**, per cui sarà necessario partire da un vettore di rendimenti attesi e da una matrice di varianza-covarianza.

In [ ]:
rng = default_rng(seed=2021)
mu2 = np.array([0.07, 0.05])
sigma = np.array([0.20, 0.15])
rho = 0.7
varcov2 = np.array([[sigma[0]**2, rho*sigma[0]*sigma[1]],
                   [rho*sigma[0]*sigma[1], sigma[1]**2]])

In [ ]:
data2 = rng.multivariate_normal(mean=mu2, cov=varcov2, size=120).transpose()

Visualizziamo quindi la distribuzione dei rendimenti tramite uno scatter plot, osservando l'effetto che ha su quest'ultima l'**alto** valore del coefficiente di **correlazione**.

In [ ]:
fig = go.Figure(go.Scatter(x=data2[0,:], y=data2[1,:], mode='markers'))
fig.update_xaxes(title='Rendimento asset 0')
fig.update_yaxes(title='Rendimento asset 1', scaleanchor = "x", scaleratio = 1)
fig.update_layout(title='Rendimenti asset con rho=' + str(rho))
fig.show()

Generiamo adesso il dataset di rendimenti del portafoglio con 10 asset. \
 Per ovvi motivi, questa volta non è possibile visualizzarne la distribuzione. In questo caso, visto il numero di dati necessari, generiamo il vettore dei rendimenti attesi e la matrice di varianza-covarianza in maniera casuale campionando da una **distribuzione uniforme**. \
  L'unico accorgimento da prendere in questo contesto è assicurarsi di generare una matrice di **varianza-covarianza** che sia **simmetrica definita positiva**; per ottenerla in maniera semplice, basta ricordare che, data una matrice $A$ qualsiasi, la matrice $A^TA$ è simmetrica semidefinita positiva. \
   Ci basterà poi solo controllare che gli autovalori siano tutti strettamente positivi. \
Infine, osserviamo che questo metodo per generare la matrice è sicuramente inadatto nel caso si vogliano creare portafogli di maggior numerosità, ma sufficientemente coveniente in questo contesto.

In [ ]:
mu10 = rng.uniform(low=0.01, high=0.20, size=10)
varcov10 = rng.uniform(low=0.05, high=0.15, size=(10,10))
varcov10 = np.matmul(varcov10.transpose(), varcov10).transpose()

In [ ]:
print('Gli autovalori della matrice di varianza-covarianza sono',np.linalg.eig(varcov10)[0])

Gli autovalori della matrice di varianza-covarianza sono [9.16451982e-01 1.78697525e-02 1.48576107e-02 5.39603907e-05
 4.07059287e-04 1.69473525e-03 3.81631781e-03 6.03821729e-03
 1.05648633e-02 1.07254487e-02]


Come osserviamo, gli autovalori sono tutti positivi. 

Analogamente a quanto fatto precedentemente, generiamo  i rendimenti.

In [ ]:
data10 = rng.multivariate_normal(mean=mu10, cov=varcov10, size=120).transpose()

Proviamo adesso a **stimare** per mezzo della media, varianza e covarianza **campionarie** i valori attesi, le varianze e le covarianze dei rendimenti del primo portafoglio utilizzando una **quantità** sempre** crescente di dati**. In particolare, usiamo prima i dati degli ultimi** 7 giorni**, poi quelli dell'ultimo mese (**30 giorni**) e infine i dati degli ultimi 4 mesi (**120 giorni**). Visualizziamo inoltre le stime per mezzo di uno scatter plot.

In [ ]:
mu_estimate7_data2 = np.mean(data2[:,-7:], axis=1)
mu_estimate30_data2 = np.mean(data2[:,-30:], axis=1)
mu_estimate120_data2 = np.mean(data2, axis=1)

In [ ]:
fig = go.Figure(go.Scatter(x=[mu2[0]], y=[mu2[1]], marker=dict(color='red', size=25), name='Dato reale'))
fig.add_trace(go.Scatter(x=[mu_estimate7_data2[0]], y=[mu_estimate7_data2[1]], marker=dict(color='green', size=15), name='Stima a 7 giorni'))
fig.add_trace(go.Scatter(x=[mu_estimate30_data2[0]], y=[mu_estimate30_data2[1]], marker=dict(color='blue', size=15), name='Stima a 1 mese'))
fig.add_trace(go.Scatter(x=[mu_estimate120_data2[0]], y=[mu_estimate120_data2[1]], marker=dict(color='magenta', size=15), name='Stima a 4 mesi'))
fig.update_xaxes(title='Rendimento asset 0')
fig.update_yaxes(scaleanchor = "x", scaleratio = 1, title='Rendimento asset 1')
fig.show()

Dallo scatterplot emerge che **stime più precise** sono associate a **quantità** di **dati** usati **maggiori**.

In [ ]:
varcov_estimate7_data2 = np.cov(data2[:,-7:])
varcov_estimate30_data2 = np.cov(data2[:,-30:])
varcov_estimate120_data2 = np.cov(data2)

Per vedere quanto buone sono le stime, questa volta non possiamo affidarci ad un plot. \ 

Proviamo invece a calcolare la **norma** 1 della **differenza** della **matrice** stima da quella originale.

In [ ]:
print('La norma 1 della differenza tra la matrice reale e la stima a 7 giorni è ' + '{0:.3}'.format(np.linalg.norm(np.subtract(varcov2, varcov_estimate7_data2), ord=1)) + '.')
print('La norma 1 della differenza tra la matrice reale e la stima a 1 mese è ' + '{0:.3}'.format(np.linalg.norm(np.subtract(varcov2, varcov_estimate30_data2), ord=1)) + '.')
print('La norma 1 della differenza tra la matrice reale e la stima a 4 mesi è ' + '{0:.3}'.format(np.linalg.norm(np.subtract(varcov2, varcov_estimate120_data2), ord=1)) + '.')

La norma 1 della differenza tra la matrice reale e la stima a 7 giorni è 0.048.
La norma 1 della differenza tra la matrice reale e la stima a 1 mese è 0.0189.
La norma 1 della differenza tra la matrice reale e la stima a 4 mesi è 0.00427.


Come è lecito aspettarsi, maggiore il numero di dati che vengono utilizzati, migliore la stima sia dei rendimenti attesi, che della matrice di varianza-covarianza. 

Facciamo adesso lo stesso per il portafoglio da 10 asset. Anche in questo caso, per visualizzare l'accuratezza delle stime, non possiamo usare uno scatter plot a causa della dimensionalità dei vettori in questione. \
 Calcoliamo quindi la norma L1 della differenza tra la stima e il dato reale. Proviamo comunque a plottare uno scatter plot in cui poniamo sull'asse x l'indice dell'asset e sull'asse y il rendimento stimato.

In [ ]:
mu_estimate7_data10 = np.mean(data10[:,-7:], axis=1)
mu_estimate30_data10 = np.mean(data10[:,-30:], axis=1)
mu_estimate120_data10 = np.mean(data10, axis=1)

print('La norma 1 della differenza tra i rendimenti attesi reali e la stima a 7 giorni è ' + '{0:.3}'.format(np.linalg.norm(np.subtract(mu10, mu_estimate7_data10), ord=1)) + '.')
print('La norma 1 della differenza tra i rendimenti attesi reali e la stima a 1 mese è ' + '{0:.3}'.format(np.linalg.norm(np.subtract(mu10, mu_estimate30_data10), ord=1)) + '.')
print('La norma 1 della differenza tra i rendimenti attesi reali e la stima a 4 mesi è ' + '{0:.3}'.format(np.linalg.norm(np.subtract(mu10, mu_estimate120_data10), ord=1)) + '.')

La norma 1 della differenza tra i rendimenti attesi reali e la stima a 7 giorni è 1.72.
La norma 1 della differenza tra i rendimenti attesi reali e la stima a 1 mese è 0.528.
La norma 1 della differenza tra i rendimenti attesi reali e la stima a 4 mesi è 0.595.


In [ ]:
assets = np.linspace(0,10,num=10,dtype=np.int8)
fig = go.Figure(go.Scatter(x=assets, y=mu10, marker=dict(color='red', size=20), name='Dato reale', mode='markers'))
fig.add_trace(go.Scatter(x=assets, y=mu_estimate7_data10, marker=dict(color='green', size=10), name='Stima a 7 giorni', mode='markers'))
fig.add_trace(go.Scatter(x=assets, y=mu_estimate30_data10, marker=dict(color='blue', size=10), name='Stima a 1 mese', mode='markers'))
fig.add_trace(go.Scatter(x=assets, y=mu_estimate120_data10, marker=dict(color='magenta', size=10), name='Stima a 4 mesi', mode='markers'))
fig.update_xaxes(title='Asset', zeroline=False, type='category')
fig.update_yaxes(title='Rendimento', showgrid=False)
fig.show()

E' interessante notare come la **stima a 7 giorni** restituisca dei **rendimenti** attesi quasi sempre **negativi**. \
 Questo fatto dipende sicuramente dai particolari dati che sono stati generati, ma è sicuramente un evento realistico (basta immaginare una settimana di mercato ribassista).

In [ ]:
varcov_estimate7_data10 = np.cov(data10[:,-7:])
varcov_estimate30_data10 = np.cov(data10[:,-30:])
varcov_estimate120_data10 = np.cov(data10)

print('La norma 1 della differenza tra la matrice reale e la stima a 7 giorni è ' + '{0:.3}'.format(np.linalg.norm(np.subtract(varcov10, varcov_estimate7_data10), ord=1)) + '.')
print('La norma 1 della differenza tra la matrice reale e la stima a 1 mese è ' + '{0:.3}'.format(np.linalg.norm(np.subtract(varcov10, varcov_estimate30_data10), ord=1)) + '.')
print('La norma 1 della differenza tra la matrice reale e la stima a 4 mesi è ' + '{0:.3}'.format(np.linalg.norm(np.subtract(varcov10, varcov_estimate120_data10), ord=1)) + '.')

La norma 1 della differenza tra la matrice reale e la stima a 7 giorni è 0.795.
La norma 1 della differenza tra la matrice reale e la stima a 1 mese è 0.259.
La norma 1 della differenza tra la matrice reale e la stima a 4 mesi è 0.0381.


Valgono anche in questo caso le considerazioni precedenti. \

Prima di proseguire, poniamo l'attenzione su una considerazione importante. Abbiamo costruito la matrice di varianza-covarianza dei rendimenti reali facendo sì che essa fosse simmetrica definita positiva. \

Invece, nel caso della **matrice campionaria**, è garantita esclusivamente la proprietà di **simmetria** ( in virtù della definizione stessa di covarianza campionaria), mentre non c'è **nessuna garanzia che essa sia anche definita positiva**. \

Calcoliamo quindi gli autovalori di tutte le matrici stimate. \
Iniziamo con il portafoglio a due asset.

In [ ]:
print('Gli autovalori della matrice di varianza-covarianza reale sono',np.linalg.eig(varcov2)[0])
print('Gli autovalori della matrice di varianza-covarianza stimata a una settimana sono',np.linalg.eig(varcov_estimate7_data2)[0])
print('Gli autovalori della matrice di varianza-covarianza stimata a un mese sono',np.linalg.eig(varcov_estimate30_data2)[0])
print('Gli autovalori della matrice di varianza-covarianza stimata a quattro mesi sono',np.linalg.eig(varcov_estimate120_data2)[0])

Gli autovalori della matrice di varianza-covarianza reale sono [0.054  0.0085]
Gli autovalori della matrice di varianza-covarianza stimata a una settimana sono [0.09575175 0.00676177]
Gli autovalori della matrice di varianza-covarianza stimata a un mese sono [0.06944777 0.00640631]
Gli autovalori della matrice di varianza-covarianza stimata a quattro mesi sono [0.05076898 0.00889556]


Passiamo adesso al portafoglio a 10 asset.

In [ ]:
print('Gli autovalori della matrice di varianza-covarianza reale sono',np.linalg.eig(varcov10)[0])
print('Gli autovalori della matrice di varianza-covarianza stimata a una settimana sono',np.linalg.eig(varcov_estimate7_data10)[0])
print('Gli autovalori della matrice di varianza-covarianza stimata a un mese sono',np.linalg.eig(varcov_estimate30_data10)[0])
print('Gli autovalori della matrice di varianza-covarianza stimata a quattro mesi sono',np.linalg.eig(varcov_estimate120_data10)[0])

Gli autovalori della matrice di varianza-covarianza reale sono [9.16451982e-01 1.78697525e-02 1.48576107e-02 5.39603907e-05
 4.07059287e-04 1.69473525e-03 3.81631781e-03 6.03821729e-03
 1.05648633e-02 1.07254487e-02]
Gli autovalori della matrice di varianza-covarianza stimata a una settimana sono [ 1.43951142e+00  2.72385938e-02  5.87887329e-03  9.63664036e-03
  1.58351838e-03  1.82615428e-04  5.32547147e-18  3.31870195e-17
 -7.22323787e-19 -2.96827424e-17]
Gli autovalori della matrice di varianza-covarianza stimata a un mese sono [7.23226101e-01 1.95267816e-02 1.12174335e-02 9.50931507e-03
 4.40834574e-03 2.77157919e-03 2.27164741e-03 1.05960234e-03
 4.73293906e-05 2.25975626e-04]
Gli autovalori della matrice di varianza-covarianza stimata a quattro mesi sono [9.36399086e-01 1.81874279e-02 1.55360278e-02 1.09965464e-02
 8.46016515e-03 5.60002641e-03 3.51828880e-03 1.39523120e-03
 5.18896920e-05 3.06215767e-04]


Osserviamo che nella **stima a una settimana** della matrice del portafoglio a 10 asset, **due autovalori** sono **negativi** ( inoltre altri quattro autovalori sono molti prossimi allo zero numerico). Quindi,  la matrice è **indefinita**. \
Questo è decisamente un **limite del modello**, che risulta in parte sbagliato, e inoltre può avere pesanti conseguenze nella risoluzione di problemi di **ottimizzazione**, in quanto, come è noto, in molti casi, l'ipotesi fondamentale è che la funzione obiettivo sia **convessa**. \

In un contesto reale sarebbe necessario trovare un modo per rendere la matrice definita positiva alterandone il meno possibile le entrate; noi ci limiteremo a osservarne alcune sue **conseguenze** (ne abbiamo un esempio lampante nella prossima sezione).

## L'effetto dell'avversione al rischio
Proviamo innanzitutto a trovare il portafoglio MV ottimo **massimizzando** il **rendimento** atteso e **minimizzando** la **varianza**. Come visto durante il corso, si può formulare tale problema di ottimizzazione multi-obiettivo introducendo un parametro $\lambda$ di **avversione** al **rischio**:
$$
\begin{align}
\max_w\ &\mu^Tw - \frac{\lambda}{2}w^T \Sigma w\\
s.t.\ & i^Tw=1
\end{align}
$$

Osserviamo che tale formulazione **consente** **vendite** allo **scoperto**. Vedremo l'impatto che ha tale assunzione successivamente. \
Implementiamo una funzione per risolvere questo problema di ottimizzazione come visto durante il corso, in modo da poterla riutilizzare sia per la soluzione teorica, sia per le diverse approssimazioni dei parametri. \
 Utilizziamo tre diversi valori per $\lambda$: 0.1, 1, 10.

In [ ]:
def risk_aversion_optimize(mu, Sigma, risk_aversion):
  i = np.ones(shape=mu.shape)
  invSigmaI = np.linalg.solve(Sigma, i) # Sigma^-1 * i
  invSigmaMu = np.linalg.solve(Sigma, mu) # Sigma^-1 * mu
  iInvSigmaI = np.dot(i,invSigmaI) # i^T * Sigma^-1 * i
  w_opt = invSigmaI/(iInvSigmaI) + (iInvSigmaI*invSigmaMu-np.dot(i,invSigmaMu)*invSigmaI)/(risk_aversion*iInvSigmaI)
  mu_opt = np.dot(mu,w_opt)
  sigma2_opt = np.dot(w_opt, Sigma.dot(w_opt))

  return w_opt, mu_opt, sigma2_opt

In [ ]:
def risk_aversion_analyze(mu, varcov):
  n = mu.shape[0]
  risk_range = 10**np.linspace(-1,1,num=3)
  w_opt_list = []
  mu_list = []
  sigma2_list = []

  for risk_aversion in risk_range:
    w_opt, mu_opt, sigma2_opt = risk_aversion_optimize(mu, varcov, risk_aversion)

    output = 'Avversione al rischio : ' + str(risk_aversion) + '\n'

    for i in range(n):
      output += '\t\tAsset ' + str(i) + ': ' + '{0:.2f}'.format(w_opt[i]) + '\n'

    output += '\t\tRendimento atteso: ' + '{0:.2f}'.format(mu_opt) + ' -- Varianza: ' + '{0:.2f}'.format(sigma2_opt) + '.'
    print(output)

    w_opt_list.append(w_opt)
    mu_list.append(mu_opt)
    sigma2_list.append(sigma2_opt)
  return w_opt_list

Iniziamo con il portafoglio a due asset e calcoliamo la **soluzione esatta,** cioè quella ottenuta utilizzando i valori reali dei rendimenti attesi e della matrice di varianza covarianza.

In [ ]:
# soluzione con dati reali
real_w_opt2 = risk_aversion_analyze(mu2, varcov2)

Avversione al rischio : 0.1
		Asset 0: 9.83
		Asset 1: -8.83
		Rendimento atteso: 0.25 -- Varianza: 1.97.
Avversione al rischio : 1.0
		Asset 0: 1.05
		Asset 1: -0.05
		Rendimento atteso: 0.07 -- Varianza: 0.04.
Avversione al rischio : 10.0
		Asset 0: 0.17
		Asset 1: 0.83
		Rendimento atteso: 0.05 -- Varianza: 0.02.


Proviamo adesso ad utilizzare i valori che abbiamo stimato su **orizzonti temporali sempre più ampi**.

In [ ]:
#stima a 1 settimana
week_w_opt2 = risk_aversion_analyze(mu_estimate7_data2, varcov_estimate7_data2)

Avversione al rischio : 0.1
		Asset 0: 49.14
		Asset 1: -48.14
		Rendimento atteso: 4.84 -- Varianza: 48.27.
Avversione al rischio : 1.0
		Asset 0: 4.62
		Asset 1: -3.62
		Rendimento atteso: 0.49 -- Varianza: 0.52.
Avversione al rischio : 10.0
		Asset 0: 0.17
		Asset 1: 0.83
		Rendimento atteso: 0.06 -- Varianza: 0.04.


In [ ]:
#stima a 1 mese
month_w_opt2 = risk_aversion_analyze(mu_estimate30_data2, varcov_estimate30_data2)

Avversione al rischio : 0.1
		Asset 0: 20.00
		Asset 1: -19.00
		Rendimento atteso: 0.83 -- Varianza: 7.75.
Avversione al rischio : 1.0
		Asset 0: 1.81
		Asset 1: -0.81
		Rendimento atteso: 0.14 -- Varianza: 0.10.
Avversione al rischio : 10.0
		Asset 0: -0.01
		Asset 1: 1.01
		Rendimento atteso: 0.07 -- Varianza: 0.02.


In [ ]:
#stima a 4 mesi
quarter_w_opt2 = risk_aversion_analyze(mu_estimate120_data2, varcov_estimate120_data2)

Avversione al rischio : 0.1
		Asset 0: 13.03
		Asset 1: -12.03
		Rendimento atteso: 0.39 -- Varianza: 3.41.
Avversione al rischio : 1.0
		Asset 0: 1.43
		Asset 1: -0.43
		Rendimento atteso: 0.09 -- Varianza: 0.06.
Avversione al rischio : 10.0
		Asset 0: 0.27
		Asset 1: 0.73
		Rendimento atteso: 0.06 -- Varianza: 0.02.


Plottiamo quindi i portafogli ottenuti.

In [ ]:
fig = make_subplots(rows=1, cols=3, horizontal_spacing=0.04, subplot_titles=['$\lambda=0.1$','$\lambda=1$','$\lambda=10$'])

for i in range(3):
  fig.add_trace(go.Scatter(x=[real_w_opt2[i][0]], y=[real_w_opt2[i][1]], marker=dict(color='red', size=25), name='Dato reale', showlegend=i==0, legendgroup=1), row=1, col=i+1)
  fig.add_trace(go.Scatter(x=[week_w_opt2[i][0]], y=[week_w_opt2[i][1]], marker=dict(color='green', size=15), name='Stima a 7 giorni', showlegend=i==0, legendgroup=2), row=1, col=i+1)
  fig.add_trace(go.Scatter(x=[month_w_opt2[i][0]], y=[month_w_opt2[i][1]], marker=dict(color='blue', size=15), name='Stima a 1 mese', showlegend=i==0, legendgroup=3), row=1, col=i+1)
  fig.add_trace(go.Scatter(x=[quarter_w_opt2[i][0]], y=[quarter_w_opt2[i][1]], marker=dict(color='magenta', size=15), name='Stima a 4 mesi', showlegend=i==0, legendgroup=4), row=1, col=i+1)
  fig.layout.annotations[i].update(y=1.03)

fig.update_xaxes(title='$w_0$', row=1)
fig.update_yaxes(scaleanchor = "x", scaleratio = 1, title='$w_1$', row=1, col=1)
fig.update_yaxes(scaleanchor = "x", scaleratio = 1)
fig.show()

Possiamo osservare innanzitutto che, utilizzando valori più **alti** di coefficienti di **avversione** al **rischio**, si tendono ad **evitare** sempre più le **vendite allo scoperto**. \
Inoltre, come è lecito aspettarsi ancora una volta, **maggiori quantità di dati** utilizzati per le stime comportano una **migliore approssimazione del portafoglio ottimo**.

Passiamo adesso al portafoglio con 10 asset.

In [ ]:
#soluzione reale
real_w_opt10 = risk_aversion_analyze(mu10, varcov10)

Avversione al rischio : 0.1
		Asset 0: -134.06
		Asset 1: 70.80
		Asset 2: -492.29
		Asset 3: 289.80
		Asset 4: 456.24
		Asset 5: -529.13
		Asset 6: -200.32
		Asset 7: 29.23
		Asset 8: 154.40
		Asset 9: 356.32
		Rendimento atteso: 106.75 -- Varianza: 1062.54.
Avversione al rischio : 1.0
		Asset 0: -13.43
		Asset 1: 5.80
		Asset 2: -48.63
		Asset 3: 29.40
		Asset 4: 49.41
		Asset 5: -53.53
		Asset 6: -22.02
		Asset 7: 1.19
		Asset 8: 16.85
		Asset 9: 35.96
		Rendimento atteso: 11.12 -- Varianza: 10.64.
Avversione al rischio : 10.0
		Asset 0: -1.37
		Asset 1: -0.69
		Asset 2: -4.27
		Asset 3: 3.36
		Asset 4: 8.73
		Asset 5: -5.97
		Asset 6: -4.19
		Asset 7: -1.62
		Asset 8: 3.10
		Asset 9: 3.92
		Rendimento atteso: 1.56 -- Varianza: 0.12.


In [ ]:
#stima a 1 settimana
week_w_opt10 = risk_aversion_analyze(mu_estimate7_data10, varcov_estimate7_data10)

Avversione al rischio : 0.1
		Asset 0: 21765739584578288.00
		Asset 1: 6869424751016368.00
		Asset 2: 20720480446735656.00
		Asset 3: -3338468312898296.50
		Asset 4: -16468755784795970.00
		Asset 5: 4037278215995251.00
		Asset 6: 28089929094620804.00
		Asset 7: -24641199043413784.00
		Asset 8: -2918204204710145.50
		Asset 9: -34116224747128176.00
		Rendimento atteso: -1483378703784543.50 -- Varianza: -170064635002910240.00.
Avversione al rischio : 1.0
		Asset 0: 2176573958457828.75
		Asset 1: 686942475101634.62
		Asset 2: 2072048044673566.75
		Asset 3: -333846831289830.19
		Asset 4: -1646875578479593.50
		Asset 5: 403727821599526.19
		Asset 6: 2808992909462078.00
		Asset 7: -2464119904341380.50
		Asset 8: -291820420471013.06
		Asset 9: -3411622474712816.50
		Rendimento atteso: -148337870378454.06 -- Varianza: -183391218810031.94.
Avversione al rischio : 10.0
		Asset 0: 217657395845782.94
		Asset 1: 68694247510161.15
		Asset 2: 207204804467357.75
		Asset 3: -33384683128983.51
		Asset 4:

In [ ]:
#stima a 1 mese
month_w_opt10 = risk_aversion_analyze(mu_estimate30_data10, varcov_estimate30_data10)

Avversione al rischio : 0.1
		Asset 0: -180.12
		Asset 1: 7.90
		Asset 2: -469.68
		Asset 3: 174.24
		Asset 4: 987.82
		Asset 5: -750.09
		Asset 6: -412.54
		Asset 7: -366.59
		Asset 8: 346.57
		Asset 9: 663.49
		Rendimento atteso: 188.92 -- Varianza: 1883.58.
Avversione al rischio : 1.0
		Asset 0: -17.52
		Asset 1: -0.67
		Asset 2: -46.40
		Asset 3: 18.48
		Asset 4: 102.58
		Asset 5: -76.03
		Asset 6: -43.33
		Asset 7: -38.71
		Asset 8: 35.64
		Asset 9: 66.97
		Rendimento atteso: 19.40 -- Varianza: 18.85.
Avversione al rischio : 10.0
		Asset 0: -1.26
		Asset 1: -1.52
		Asset 2: -4.07
		Asset 3: 2.90
		Asset 4: 14.05
		Asset 5: -8.63
		Asset 6: -6.41
		Asset 7: -5.92
		Asset 8: 4.54
		Asset 9: 7.31
		Rendimento atteso: 2.45 -- Varianza: 0.20.


In [ ]:
#stima a 4 mesi
quarter_w_opt10 = risk_aversion_analyze(mu_estimate120_data10, varcov_estimate120_data10)

Avversione al rischio : 0.1
		Asset 0: -95.79
		Asset 1: 39.37
		Asset 2: -444.45
		Asset 3: 123.25
		Asset 4: 570.10
		Asset 5: -381.07
		Asset 6: -187.06
		Asset 7: -165.55
		Asset 8: 305.21
		Asset 9: 236.99
		Rendimento atteso: 112.39 -- Varianza: 1118.44.
Avversione al rischio : 1.0
		Asset 0: -9.50
		Asset 1: 2.54
		Asset 2: -43.97
		Asset 3: 13.03
		Asset 4: 60.86
		Asset 5: -38.90
		Asset 6: -20.82
		Asset 7: -18.38
		Asset 8: 31.88
		Asset 9: 24.27
		Rendimento atteso: 11.73 -- Varianza: 11.20.
Avversione al rischio : 10.0
		Asset 0: -0.87
		Asset 1: -1.15
		Asset 2: -3.93
		Asset 3: 2.01
		Asset 4: 9.93
		Asset 5: -4.68
		Asset 6: -4.20
		Asset 7: -3.66
		Asset 8: 4.54
		Asset 9: 3.00
		Rendimento atteso: 1.67 -- Varianza: 0.12.


Osserviamo che con un **portafoglio più numeroso** la stima a una settimana è veramente **pessima**.\
 Una **possibile spiegazione** è, come osservato in precedenza, che la matrice di varianza-covarianza stimata con i dati di una settimana **non** è **definita** **positiva**. Come conseguenza, vettori di pesi, $w$, per cui $ w^T \Sigma w <0 $, fanno crescere  il valore della funzione obiettivo ( dato il segno negativo di fronte al termine). \

 Plottiamo nuovamente i risultati, evitando, per una maggiore chiarezza rappresentativa, di mostrare proprio le stime a una settimana.

In [ ]:
assets = np.linspace(0,10,num=10,dtype=np.int8)
fig = make_subplots(cols=1, rows=3, vertical_spacing=0.06, subplot_titles=['$\lambda=0.1$','$\lambda=1$','$\lambda=10$'])

for i in range(3):
  fig.add_trace(go.Scatter(x=assets, y=real_w_opt10[i], marker=dict(color='red', size=20), name='Dato reale', mode='markers', showlegend=i==0, legendgroup=1), col=1, row=i+1)
  fig.add_trace(go.Scatter(x=assets, y=month_w_opt10[i], marker=dict(color='blue', size=10), name='Stima a 1 mese', mode='markers', showlegend=i==0, legendgroup=2), col=1, row=i+1)
  fig.add_trace(go.Scatter(x=assets, y=quarter_w_opt10[i], marker=dict(color='magenta', size=10), name='Stima a 4 mesi', mode='markers', showlegend=i==0, legendgroup=3), col=1, row=i+1)

fig.update_xaxes(title='Asset', zeroline=False, type='category')
fig.update_yaxes(title='$w$', showgrid=False)
fig.update_layout(height=1500)
fig.show()

Possiamo trarre anche in questo caso le stesse conclusioni ottenute in precedenza.\
 E', inoltre, interessante notare che, al di là della bontà del risultato ottenuto, quasi sempre gli **asset** che andrebbero **venduti allo scoperto** secondo la strategia ottima basata su **dati reali** sono venduti allo scoperto **anche** basandosi sulle **approssimazioni**. 

Per avere una migliore idea della stima ottenuta, misuriamo l'errore, calcolando la norma 1 della differenza tra le varie stime e il risultato esatto. Per tenere conto della **diversa numerosità** dei due portafogli, **dividiamo** la norma per il **numero di asset** nel portafoglio. \

Per prima cosa, vediamo i risultati riguardanti il portafoglio a 2 asset.

In [ ]:
print('Stima a 1 settimana:\n\tAvversione al rischio:0.1\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt2[0]-week_w_opt2[0], ord=1)/2))
print('\tAvversione al rischio:1\t\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt2[1]-week_w_opt2[1], ord=1)/2))
print('\tAvversione al rischio:10\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt2[2]-week_w_opt2[2], ord=1)/2))
print('Stima a 1 mese:\n\tAvversione al rischio:0.1\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt2[0]-month_w_opt2[0], ord=1)/2))
print('\tAvversione al rischio:1\t\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt2[1]-month_w_opt2[1], ord=1)/2))
print('\tAvversione al rischio:10\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt2[2]-month_w_opt2[2], ord=1)/2))
print('Stima a 4 mesi:\n\tAvversione al rischio:0.1\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt2[0]-quarter_w_opt2[0], ord=1)/2))
print('\tAvversione al rischio:1\t\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt2[1]-quarter_w_opt2[1], ord=1)/2))
print('\tAvversione al rischio:10\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt2[2]-quarter_w_opt2[2], ord=1)/2))

Stima a 1 settimana:
	Avversione al rischio:0.1	Errore: 39.3105
	Avversione al rischio:1		Errore: 3.5708
	Avversione al rischio:10	Errore: 0.0031
Stima a 1 mese:
	Avversione al rischio:0.1	Errore: 10.1732
	Avversione al rischio:1		Errore: 0.7581
	Avversione al rischio:10	Errore: 0.1834
Stima a 4 mesi:
	Avversione al rischio:0.1	Errore: 3.2020
	Avversione al rischio:1		Errore: 0.3840
	Avversione al rischio:10	Errore: 0.1022


E' immediato notare che valori **maggiori** del fattore di **avversione** al **rischio** permettano di **contenere** gli **errori** sulla stima dei portafogli ottimi. \

Come vedremo successivamente, **errori** sulla **stima dei rendimenti attesi** **pesano** di **più** ( rispetto a errori sulla stima della matrice di varianza-covarianza) sulla correttezza del risultato. \
Infatti, poiché all'**aumentare di lambda**, **cresce** il **peso** nella funzione obiettivo della **matrice di varianza-covarianza**  rispetto ai rendimenti attesi, l'**errore** commesso sarà **minore**. 
Un caso estremo è il **portafoglio a minima varianza**, in cui il peso dei rendimenti attesi è nullo.

Passiamo ora al portafoglio a 10 asset.

In [ ]:
print('Stima a 1 settimana:\n\tAvversione al rischio:0.1\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt10[0]-week_w_opt10[0], ord=1)/10))
print('\tAvversione al rischio:1\t\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt10[1]-week_w_opt10[1], ord=1)/10))
print('\tAvversione al rischio:10\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt10[2]-week_w_opt10[2], ord=1)/10))
print('Stima a 1 mese:\n\tAvversione al rischio:0.1\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt10[0]-month_w_opt10[0], ord=1)/10))
print('\tAvversione al rischio:1\t\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt10[1]-month_w_opt10[1], ord=1)/10))
print('\tAvversione al rischio:10\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt10[2]-month_w_opt10[2], ord=1)/10))
print('Stima a 4 mesi:\n\tAvversione al rischio:0.1\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt10[0]-quarter_w_opt10[0], ord=1)/10))
print('\tAvversione al rischio:1\t\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt10[1]-quarter_w_opt10[1], ord=1)/10))
print('\tAvversione al rischio:10\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt10[2]-quarter_w_opt10[2], ord=1)/10))

Stima a 1 settimana:
	Avversione al rischio:0.1	Errore: 16296570418589532.0000
	Avversione al rischio:1		Errore: 1629657041858953.5000
	Avversione al rischio:10	Errore: 162965704185895.4375
Stima a 1 mese:
	Avversione al rischio:0.1	Errore: 210.7046
	Avversione al rischio:1		Errore: 21.0397
	Avversione al rischio:10	Errore: 2.0939
Stima a 4 mesi:
	Avversione al rischio:0.1	Errore: 102.4184
	Avversione al rischio:1		Errore: 10.1763
	Avversione al rischio:10	Errore: 0.9551


Anche in questo caso una maggiore avversione al rischio permette di contenere maggiormente gli errori sui portafogli ottimi ottenuti. \
 E' interessante notare che in questo caso la relazione tra $\lambda$ e gli errori è quasi esatta: ad un **aumento di $\lambda$ di 10 volte**, corrisponde una **riduzione** dell' **errore di circa 10 volte** (e di 100 volte della norma L1).

## Il portafoglio a minima varianza
Calcoliamo adesso il portafoglio a **minima varianza**, risolviamo cioè il seguente problema di ottimizzazione:
$$
\begin{align}
\min_w\ &\frac{1}{2} w^T\Sigma w^T\\
s.t.\ &i^T w = 1
\end{align}
$$

Osserviamo nuovamente la possibilità di **vendite allo scoperto**. Inoltre, possiamo considerare questo come il caso limite del problema di ottimizzazione precedente, quando $\lambda \rightarrow +\infty$. \
Definiamo nuovamente delle funzioni per risolvere il problema.

In [ ]:
def min_variance_optimize(mu, Sigma):
  i = np.ones(shape=mu.shape)
  invSigmaI = np.linalg.solve(Sigma, i) # Sigma^-1 * i
  iInvSigmaI = np.dot(i,invSigmaI) # i^T * Sigma^-1 * i
  w_opt = invSigmaI/(iInvSigmaI)
  mu_opt = np.dot(mu,w_opt)
  sigma2_opt = np.dot(w_opt, Sigma.dot(w_opt))

  return w_opt, mu_opt, sigma2_opt

def min_variance_analyze(mu, varcov):
  n = mu.shape[0]

  w_opt, mu_opt, sigma2_opt = min_variance_optimize(mu, varcov)
  
  output = ''
  for i in range(n):
    output += 'Asset ' + str(i) + ': ' + '{0:.2f}'.format(w_opt[i]) + '\n'

  output += 'Rendimento atteso: ' + '{0:.2f}'.format(mu_opt) + ' -- Varianza: ' + '{0:.2f}'.format(sigma2_opt) + '.'
  print(output)

  return w_opt

Iniziamo con il portafoglio a 2 asset.

In [ ]:
# risultato reale
real_w_opt2 = min_variance_analyze(mu2, varcov2)

Asset 0: 0.07
Asset 1: 0.93
Rendimento atteso: 0.05 -- Varianza: 0.02.


In [ ]:
# stima a 1 settimana
week_w_opt2 = min_variance_analyze(mu_estimate7_data2, varcov_estimate7_data2)

Asset 0: -0.33
Asset 1: 1.33
Rendimento atteso: 0.01 -- Varianza: 0.03.


In [ ]:
# stima a 1 mese
month_w_opt2 = min_variance_analyze(mu_estimate30_data2, varcov_estimate30_data2)

Asset 0: -0.21
Asset 1: 1.21
Rendimento atteso: 0.06 -- Varianza: 0.02.


In [ ]:
# stima a 4 mesi
quarter_w_opt2 = min_variance_analyze(mu_estimate120_data2, varcov_estimate120_data2)

Asset 0: 0.14
Asset 1: 0.86
Rendimento atteso: 0.05 -- Varianza: 0.02.


Notiamo  **risultati meno estremi** rispetto a quelli ottenuti con l'approccio precedente. \

Ciò non  sorprende se consideriamo questo approccio, ancora una volta, come un caso particolare di quello della sezione precedente, in cui $\lambda \rightarrow +\infty$ e se ricordiamo che valori maggiori del fattore di avversione al rischio comportano minori errori di stima dei portafogli ottimi. \
Vediamo ancora una volta una rappresentazione delle stime appena ottenute.

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=[real_w_opt2[0]], y=[real_w_opt2[1]], marker=dict(color='red', size=25), name='Dato reale'))
fig.add_trace(go.Scatter(x=[week_w_opt2[0]], y=[week_w_opt2[1]], marker=dict(color='green', size=15), name='Stima a 7 giorni'))
fig.add_trace(go.Scatter(x=[month_w_opt2[0]], y=[month_w_opt2[1]], marker=dict(color='blue', size=15), name='Stima a 1 mese'))
fig.add_trace(go.Scatter(x=[quarter_w_opt2[0]], y=[quarter_w_opt2[1]], marker=dict(color='magenta', size=15), name='Stima a 4 mesi'))

fig.update_xaxes(title='$w_0$')
fig.update_yaxes(scaleanchor = "x", scaleratio = 1, title='$w_1$')
fig.show()

Passiamo adesso al portafoglio a 10 asset.

In [ ]:
real_w_opt10 = min_variance_analyze(mu10, varcov10)

Asset 0: -0.03
Asset 1: -1.42
Asset 2: 0.66
Asset 3: 0.47
Asset 4: 4.21
Asset 5: -0.69
Asset 6: -2.20
Asset 7: -1.93
Asset 8: 1.57
Asset 9: 0.36
Rendimento atteso: 0.50 -- Varianza: 0.02.


In [ ]:
week_w_opt10 = min_variance_analyze(mu_estimate7_data10, varcov_estimate7_data10)

Asset 0: 0.06
Asset 1: -2.56
Asset 2: 1.19
Asset 3: -0.55
Asset 4: 4.09
Asset 5: 1.13
Asset 6: -2.80
Asset 7: -2.60
Asset 8: 1.64
Asset 9: 1.40
Rendimento atteso: 0.31 -- Varianza: -0.00.


In [ ]:
month_w_opt10 = min_variance_analyze(mu_estimate30_data10, varcov_estimate30_data10)

Asset 0: 0.54
Asset 1: -1.62
Asset 2: 0.64
Asset 3: 1.17
Asset 4: 4.22
Asset 5: -1.14
Asset 6: -2.31
Asset 7: -2.28
Asset 8: 1.09
Asset 9: 0.69
Rendimento atteso: 0.57 -- Varianza: 0.01.


In [ ]:
quarter_w_opt10 = min_variance_analyze(mu_estimate120_data10, varcov_estimate120_data10)

Asset 0: 0.09
Asset 1: -1.56
Asset 2: 0.52
Asset 3: 0.78
Asset 4: 4.27
Asset 5: -0.88
Asset 6: -2.35
Asset 7: -2.02
Asset 8: 1.51
Asset 9: 0.64
Rendimento atteso: 0.55 -- Varianza: 0.01.


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=assets, y=real_w_opt10, marker=dict(color='red', size=20), name='Dato reale', mode='markers'))
fig.add_trace(go.Scatter(x=assets, y=week_w_opt10, marker=dict(color='green', size=10), name='Stima a 7 giorni', mode='markers'))
fig.add_trace(go.Scatter(x=assets, y=month_w_opt10, marker=dict(color='blue', size=10), name='Stima a 1 mese', mode='markers'))
fig.add_trace(go.Scatter(x=assets, y=quarter_w_opt10, marker=dict(color='magenta', size=10), name='Stima a 4 mesi', mode='markers'))
  
fig.update_xaxes(title='Asset', zeroline=False, type='category')
fig.update_yaxes(title='$w$', showgrid=False)
fig.show()

Stiamiamo di nuovo gli errori nella stima dei pesi calcolando la norma 1 della differenza tra il risultato reale e quello approssimato. \
Nel caso del portafoglio con due asset abbiamo i seguenti risultati.

In [ ]:
print('Stima a 1 settimana:\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt2-week_w_opt2, ord=1)/2))
print('Stima a 1 mese:\t\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt2-month_w_opt2, ord=1)/2))
print('Stima a 4 mesi:\t\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt2-quarter_w_opt2, ord=1)/2))

Stima a 1 settimana:	Errore: 0.4003
Stima a 1 mese:		Errore: 0.2880
Stima a 4 mesi:		Errore: 0.0709


Invece, i risultati nel caso del portafoglio con 10 asset sono i seguenti.

In [ ]:
print('Stima a 1 settimana:\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt10-week_w_opt10, ord=1)/10))
print('Stima a 1 mese:\t\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt10-month_w_opt10, ord=1)/10))
print('Stima a 4 mesi:\t\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt10-quarter_w_opt10, ord=1)/10))

Stima a 1 settimana:	Errore: 0.7088
Stima a 1 mese:		Errore: 0.3216
Stima a 4 mesi:		Errore: 0.1556


Come possiamo osservare, l'approccio a **minima varianza** permette di **ridurre** di molto l'**errore di stima** dei portafoglio ottimi rispetto all'approccio MV.

## L'effetto dei vincoli sulle vendite allo scoperto

Nelle precedenti sezioni, abbiamo ammesso la possibilità di avere dei pesi negativi, cioè di vendere allo scoperto alcuni asset.\
 Proviamo adesso a escludere questa opzione e risolviamo  il seguente problema:
$$
\begin{align}
\min_w\ &\frac{1}{2} w^T\Sigma w^T\\
s.t.\ &i^T w = 1\\
&w_i \geq 0 \quad \forall i
\end{align}
$$

Durante il corso, non abbiamo visto una soluzione analitica a questo problema.  Serviamoci, quindi, della libreria *quadprog* di Python, utile  a risolvere problemi QP.  \
Quindi, definiamo  due funzioni per risolvere il problema di ottimizzazione e per visualizzarne meglio i risultati.

In [ ]:
def quadprog_solve_qp(P, q, G=None, h=None, A=None, b=None):
    qp_G = P
    qp_a = -q
    if A is not None:
        qp_C = -np.vstack([A, G]).T
        qp_b = -np.hstack([b, h])
        meq = A.shape[0]
    else:  # no equality constraint
        qp_C = -G.T
        qp_b = -h
        meq = 0
    return quadprog.solve_qp(qp_G, qp_a, qp_C, qp_b, meq)[0]

def no_short_sales_analyze(w_opt, mu, varcov):
  n = w_opt.shape[0]
  mu_opt = np.dot(mu,w_opt)
  sigma2_opt = np.dot(w_opt, varcov.dot(w_opt))

  output = ''
  for i in range(n):
    output += 'Asset ' + str(i) + ': ' + '{0:.2f}'.format(w_opt[i]) + '\n'

  output += 'Rendimento atteso: ' + '{0:.2f}'.format(mu_opt) + ' -- Varianza: ' + '{0:.2f}'.format(sigma2_opt) + '.'
  print(output)

  return w_opt

Iniziamo ancora una volta dal portafoglio a due asset.

In [ ]:
q = np.array([0,0], dtype=np.double) # non abbiamo la componente lineare
G = -np.identity(2,dtype=np.double) # matrice per vincoli di disuguaglianza
h = np.array([0,0],dtype=np.double) # vettore per vincoli di disuguaglianza
A = np.array([[1,1]],dtype=np.double) # matrice per vincoli di uguaglianza
b = np.array(1,dtype=np.double) # vettore per vincoli di uguaglianza

#risultato reale
real_minvar_w_opt2 = no_short_sales_analyze(quadprog_solve_qp(varcov2, q, G, h, A, b), mu2, varcov2)

Asset 0: 0.07
Asset 1: 0.93
Rendimento atteso: 0.05 -- Varianza: 0.02.


In [ ]:
#stima a 1 settimana
week_minvar_w_opt2 = no_short_sales_analyze(quadprog_solve_qp(varcov_estimate7_data2, q, G, h, A, b), mu_estimate7_data2, varcov_estimate7_data2)

Asset 0: 0.00
Asset 1: 1.00
Rendimento atteso: 0.04 -- Varianza: 0.03.


In [ ]:
#stima a 1 mese
month_minvar_w_opt2 = no_short_sales_analyze(quadprog_solve_qp(varcov_estimate30_data2, q, G, h, A, b), mu_estimate30_data2, varcov_estimate30_data2)

Asset 0: 0.00
Asset 1: 1.00
Rendimento atteso: 0.07 -- Varianza: 0.02.


In [ ]:
#stima a 4 mesi
quarter_minvar_w_opt2 = no_short_sales_analyze(quadprog_solve_qp(varcov_estimate120_data2, q, G, h, A, b), mu_estimate120_data2, varcov_estimate120_data2)

Asset 0: 0.14
Asset 1: 0.86
Rendimento atteso: 0.05 -- Varianza: 0.02.


In questo caso si notano sicuramente dei **risultati più accurati** anche con **pochi dati**.

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=[real_minvar_w_opt2[0]], y=[real_minvar_w_opt2[1]], marker=dict(color='red', size=25), name='Dato reale'))
fig.add_trace(go.Scatter(x=[week_minvar_w_opt2[0]], y=[week_minvar_w_opt2[1]], marker=dict(color='green', size=15), name='Stima a 7 giorni'))
fig.add_trace(go.Scatter(x=[month_minvar_w_opt2[0]], y=[month_minvar_w_opt2[1]], marker=dict(color='blue', size=15), name='Stima a 1 mese'))
fig.add_trace(go.Scatter(x=[quarter_minvar_w_opt2[0]], y=[quarter_minvar_w_opt2[1]], marker=dict(color='magenta', size=15), name='Stima a 4 mesi'))

fig.update_xaxes(title='$w_0$')
fig.update_yaxes(scaleanchor = "x", scaleratio = 1, title='$w_1$')
fig.show()

Passiamo adesso al portafoglio a 10 asset. \
Ricordiamo che la matrice di varianza-covarianza stimata con i dati di una settimana non è definita positiva, per questa ragione il solver non è in grado di trovare una soluzione.

In [ ]:
q = np.zeros(shape=10, dtype=np.double) # non abbiamo la componente lineare
G = -np.identity(10,dtype=np.double) # matrice per vincoli di disuguaglianza
h = np.zeros(shape=10,dtype=np.double) # vettore per vincoli di disuguaglianza
A = np.ones(shape=[1,10],dtype=np.double) # matrice per vincoli di uguaglianza
b = np.array(1,dtype=np.double) # vettore per vincoli di uguaglianza

# risultato reale
real_minvar_w_opt10 = no_short_sales_analyze(quadprog_solve_qp(varcov10, q, G, h, A, b), mu10, varcov10)

Asset 0: 0.00
Asset 1: 0.00
Asset 2: 0.14
Asset 3: 0.12
Asset 4: 0.74
Asset 5: -0.00
Asset 6: 0.00
Asset 7: 0.00
Asset 8: -0.00
Asset 9: 0.00
Rendimento atteso: 0.16 -- Varianza: 0.07.


In [ ]:
# stima a 1 mese
month_minvar_w_opt10 = no_short_sales_analyze(quadprog_solve_qp(varcov_estimate30_data10, q, G, h, A, b), mu_estimate30_data10, varcov_estimate30_data10)

Asset 0: 0.00
Asset 1: 0.00
Asset 2: 0.00
Asset 3: 0.65
Asset 4: 0.35
Asset 5: -0.00
Asset 6: 0.00
Asset 7: -0.00
Asset 8: -0.00
Asset 9: -0.00
Rendimento atteso: 0.11 -- Varianza: 0.05.


In [ ]:
# stima a 4 mesi
quarter_minvar_w_opt10 = no_short_sales_analyze(quadprog_solve_qp(varcov_estimate120_data10, q, G, h, A, b), mu_estimate120_data10, varcov_estimate120_data10)

Asset 0: 0.00
Asset 1: -0.00
Asset 2: 0.00
Asset 3: 0.35
Asset 4: 0.65
Asset 5: 0.00
Asset 6: -0.00
Asset 7: -0.00
Asset 8: -0.00
Asset 9: -0.00
Rendimento atteso: 0.11 -- Varianza: 0.07.


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=assets, y=real_minvar_w_opt10, marker=dict(color='red', size=20), name='Dato reale', mode='markers'))
fig.add_trace(go.Scatter(x=assets, y=month_minvar_w_opt10, marker=dict(color='blue', size=10), name='Stima a 1 mese', mode='markers'))
fig.add_trace(go.Scatter(x=assets, y=quarter_minvar_w_opt10, marker=dict(color='magenta', size=10), name='Stima a 4 mesi', mode='markers'))
  
fig.update_xaxes(title='Asset', zeroline=False, type='category')
fig.update_yaxes(title='$w$', showgrid=False)
fig.show()

In questo caso, ciò che colpisce maggiormente è che **alcuni asset** (quelli con maggiore varianza), **piuttosto che** essere **venduti allo scoperto**, **non** vengono **acquistati**. Questa decisione viene presa anche affidandosi a **pochi dati**. \
**Invece**, a **pesi strettamente positivi**, corrispondono **stime meno precise**.



Proviamo adesso a introdurre il **vincolo** sulle **vendite allo scoperto** anche al **portafoglio MV**:
$$
\begin{align}
\max_w\ &\mu^Tw - \frac{\lambda}{2}w^T \Sigma w\\
s.t.\ & i^Tw=1 \\
& w_i \geq 0 \quad \forall i
\end{align}
$$

In questo caso, fissiamo $\lambda = 0.1$. Scegliamo questo valore in quanto è quello che amplificava maggiormente gli errori di stima tra quelli usati nella prima sezione. Per utilizzare il solver, dobbiamo prima riscrivere il problema in forma di minimizzazione, cioè:
$$
\begin{align}
\min_w\ &\frac{\lambda}{2} w^T \Sigma w - \mu^T w\\
s.t.\ & i^Tw=1 \\
& w_i \geq 0 \quad \forall i
\end{align}
$$

Iniziamo  dal portafoglio con due asset.

In [ ]:
G = -np.identity(2,dtype=np.double) # matrice per vincoli di disuguaglianza
h = np.array([0,0],dtype=np.double) # vettore per vincoli di disuguaglianza
A = np.array([[1,1]],dtype=np.double) # matrice per vincoli di uguaglianza
b = np.array(1,dtype=np.double) # vettore per vincoli di uguaglianza

#risultato reale
real_w_opt2 = no_short_sales_analyze(quadprog_solve_qp(0.1*varcov2, -mu2, G, h, A, b), mu2, varcov2)

Asset 0: 1.00
Asset 1: 0.00
Rendimento atteso: 0.07 -- Varianza: 0.04.


In [ ]:
#stima a 1 settimana
week_w_opt2 = no_short_sales_analyze(quadprog_solve_qp(0.1*varcov_estimate7_data2, -mu_estimate7_data2, G, h, A, b), mu_estimate7_data2, varcov_estimate7_data2)

Asset 0: 1.00
Asset 1: 0.00
Rendimento atteso: 0.14 -- Varianza: 0.07.


In [ ]:
#stima a 1 mese
month_w_opt2 = no_short_sales_analyze(quadprog_solve_qp(0.1*varcov_estimate30_data2, -mu_estimate30_data2, G, h, A, b), mu_estimate30_data2, varcov_estimate30_data2)

Asset 0: 1.00
Asset 1: 0.00
Rendimento atteso: 0.10 -- Varianza: 0.05.


In [ ]:
#stima a 4 mesi
quarter_w_opt2 = no_short_sales_analyze(quadprog_solve_qp(0.1*varcov_estimate120_data2, -mu_estimate120_data2, G, h, A, b), mu_estimate120_data2, varcov_estimate120_data2)

Asset 0: 1.00
Asset 1: 0.00
Rendimento atteso: 0.08 -- Varianza: 0.04.


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=[real_w_opt2[0]], y=[real_w_opt2[1]], marker=dict(color='red', size=25), name='Dato reale'))
fig.add_trace(go.Scatter(x=[week_w_opt2[0]], y=[week_w_opt2[1]], marker=dict(color='green', size=15), name='Stima a 7 giorni'))
fig.add_trace(go.Scatter(x=[month_w_opt2[0]], y=[month_w_opt2[1]], marker=dict(color='blue', size=15), name='Stima a 1 mese'))
fig.add_trace(go.Scatter(x=[quarter_w_opt2[0]], y=[quarter_w_opt2[1]], marker=dict(color='magenta', size=15), name='Stima a 4 mesi'))

fig.update_xaxes(title='$w_0$')
fig.update_yaxes(scaleanchor = "x", scaleratio = 1, title='$w_1$')
fig.show()

Questa volta **tutte le stime sono corrette**: compriamo **solo** l'**asset** con **rendimento** atteso **maggiore**. \
Passiamo al portafoglio con 10 asset.

In [ ]:
G = -np.identity(10,dtype=np.double) # matrice per vincoli di disuguaglianza
h = np.zeros(shape=10,dtype=np.double) # vettore per vincoli di disuguaglianza
A = np.ones(shape=[1,10],dtype=np.double) # matrice per vincoli di uguaglianza
b = np.array(1,dtype=np.double) # vettore per vincoli di uguaglianza

#risultato reale
real_w_opt10 = no_short_sales_analyze(quadprog_solve_qp(0.1*varcov10, -mu10, G, h, A, b), mu10, varcov10)

Asset 0: 0.00
Asset 1: -0.00
Asset 2: 0.00
Asset 3: 0.00
Asset 4: 0.00
Asset 5: -0.00
Asset 6: -0.00
Asset 7: -0.00
Asset 8: 0.00
Asset 9: 1.00
Rendimento atteso: 0.18 -- Varianza: 0.11.


In [ ]:
#stima a 1 mese
month_w_opt10 = no_short_sales_analyze(quadprog_solve_qp(0.1*varcov_estimate30_data10, -mu_estimate30_data10, G, h, A, b), mu_estimate30_data10, varcov_estimate30_data10)

Asset 0: 0.00
Asset 1: 0.00
Asset 2: 0.00
Asset 3: 0.00
Asset 4: -0.00
Asset 5: 0.00
Asset 6: 0.00
Asset 7: -0.00
Asset 8: 1.00
Asset 9: -0.00
Rendimento atteso: 0.13 -- Varianza: 0.08.


In [ ]:
#stima a 4 mesi
quarter_w_opt10 = no_short_sales_analyze(quadprog_solve_qp(0.1*varcov_estimate120_data10, -mu_estimate120_data10, G, h, A, b), mu_estimate120_data10, varcov_estimate120_data10)

Asset 0: -0.00
Asset 1: 0.00
Asset 2: -0.00
Asset 3: -0.00
Asset 4: 0.00
Asset 5: -0.00
Asset 6: -0.00
Asset 7: 0.00
Asset 8: -0.00
Asset 9: 1.00
Rendimento atteso: 0.14 -- Varianza: 0.11.


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=assets, y=real_w_opt10, marker=dict(color='red', size=20), name='Dato reale', mode='markers'))
fig.add_trace(go.Scatter(x=assets, y=month_w_opt10, marker=dict(color='blue', size=10), name='Stima a 1 mese', mode='markers'))
fig.add_trace(go.Scatter(x=assets, y=quarter_w_opt10, marker=dict(color='magenta', size=10), name='Stima a 4 mesi', mode='markers'))
  
fig.update_xaxes(title='Asset', zeroline=False, type='category')
fig.update_yaxes(title='$w$', showgrid=False)
fig.show()

Anche in questo caso,  i **pesi** di molti asset sono **nulli** sia nella soluzione esatta che in tutte le approssimazioni, cioè si preferisce un **portafoglio poco diverficato**.  \
La stima ad un mese, però, porta ad acquistare solamente l'asset 'sbagliato'.

Calcoliamo nuovamente gli errori in norma 1 delle stime. Iniziamo dal portafoglio a due asset ottenuto minimizzando la sola varianza.

In [ ]:
print('Stima a 1 settimana:\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_minvar_w_opt2-week_minvar_w_opt2, ord=1)/2))
print('Stima a 1 mese:\t\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_minvar_w_opt2-month_minvar_w_opt2, ord=1)/2))
print('Stima a 4 mesi:\t\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_minvar_w_opt2-quarter_minvar_w_opt2, ord=1)/2))

Stima a 1 settimana:	Errore: 0.0732
Stima a 1 mese:		Errore: 0.0732
Stima a 4 mesi:		Errore: 0.0709


Vediamo adesso gli errori dei portafogli ottenuti introducendo il fattore di avversione al rischio.

In [ ]:
print('Stima a 1 settimana:\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt2-week_w_opt2, ord=1)/2))
print('Stima a 1 mese:\t\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt2-month_w_opt2, ord=1)/2))
print('Stima a 4 mesi:\t\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt2-quarter_w_opt2, ord=1)/2))

Stima a 1 settimana:	Errore: 0.0000
Stima a 1 mese:		Errore: 0.0000
Stima a 4 mesi:		Errore: 0.0000


Facciamo lo stesso per il portafoglio a 10 asset. Gli errori ottenuti minimizzando la sola varianza sono i seguenti.

In [ ]:
print('Stima a 1 mese:\t\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_minvar_w_opt10-month_minvar_w_opt10, ord=1)/10))
print('Stima a 4 mesi:\t\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_minvar_w_opt10-quarter_minvar_w_opt10, ord=1)/10))

Stima a 1 mese:		Errore: 0.1064
Stima a 4 mesi:		Errore: 0.0465


Vediamo adesso gli errori dei portafogli ottenuti introducendo il fattore di avversione al rischio.

In [ ]:
print('Stima a 1 mese:\t\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt10-month_w_opt10, ord=1)/10))
print('Stima a 4 mesi:\t\tErrore: ' + '{0:.4f}'.format(np.linalg.norm(real_w_opt10-quarter_w_opt10, ord=1)/10))

Stima a 1 mese:		Errore: 0.2000
Stima a 4 mesi:		Errore: 0.0000


L'aggiunta del **vincolo sulle vendite allo scoperto** sembra avere un **ottimo effetto sulla stima** dei portafogli ottimi in ogni caso. \
 Un netto miglioramento, in particolare, si può osservare sui **portafogli MV**, in cui l'**errore** è, in molti casi, **azzerato**.  Questo miglioramento può essere spiegato pensando al caso in cui un **rendimento** atteso viene **sottostimato**: il portafoglio ottimo tenderà a **vendere** il relativo asset allo **scoperto**, causando un errore nella stima dei pesi ottimi.\
  Introducendo il vincolo sulle vendite allo scoperto, invece, questo errore viene evitato.

## Rendimenti attesi o covarianze? Quali errori pesano di più?
Come abbiamo già intuito in precedenza, gli **errori sulla stima dei rendimenti attesi sembrano pesare di più degli errori sulla matrice di varianza-covarianza**. Ricordiamo, però, che errori sulla stima della matrice di varianza covarianza possono portare quest'ultima a non essere definita positiva. \
Per studiare meglio il fenomeno, risolviamo di nuovo i problemi risolti in precedenza, ma isolando le due stime e cioè:
- nel primo caso,  forniamo il vettore dei **rendimenti esatti** e la matrice di **varianza-covarianza stimata** ;
- nel secondo caso, utilizziamo il vettore dei **rendimenti stimati** e la matrice di **varianza-covarianza esatta**.

Quindi quantifichiamo l'errore nei due casi distinti. \
Rimuoviamo i vincoli sulle vendite allo scoperto, che, come abbiamo visto, hanno l'effetto di ridurre gli errori di stima. \


Calcoliamo quindi gli **errori** nelle stime dei portafogli a **minima varianza**.

In [ ]:
## PORTAFOGLIO A 2 Asset
# risultato reale
real_minvar_w_opt2, _, _ = min_variance_optimize(mu2, varcov2)

# stime con rendimenti attesi reali
week_realmu_minvar_w_opt2, _, _ = min_variance_optimize(mu2, varcov_estimate7_data2)
month_realmu_minvar_w_opt2, _, _ = min_variance_optimize(mu2, varcov_estimate30_data2)
quarter_realmu_minvar_w_opt2, _, _ = min_variance_optimize(mu2, varcov_estimate120_data2)

# stime con matrice di varianza-covarianza reale
week_realvar_minvar_w_opt2, _, _ = min_variance_optimize(mu_estimate7_data2, varcov2)
month_realvar_minvar_w_opt2, _, _ = min_variance_optimize(mu_estimate30_data2, varcov2)
quarter_realvar_minvar_w_opt2, _, _ = min_variance_optimize(mu_estimate120_data2, varcov2)


## PORTAFOGLIO A 10 Asset
# risultato reale
real_minvar_w_opt10, _, _ = min_variance_optimize(mu10, varcov10)

# stime con rendimenti attesi reali
week_realmu_minvar_w_opt10, _, _ = min_variance_optimize(mu10, varcov_estimate7_data10)
month_realmu_minvar_w_opt10, _, _ = min_variance_optimize(mu10, varcov_estimate30_data10)
quarter_realmu_minvar_w_opt10, _, _ = min_variance_optimize(mu10, varcov_estimate120_data10)

# stime con matrice di varianza-covarianza reale
week_realvar_minvar_w_opt10, _, _ = min_variance_optimize(mu_estimate7_data10, varcov10)
month_realvar_minvar_w_opt10, _, _ = min_variance_optimize(mu_estimate30_data10, varcov10)
quarter_realvar_minvar_w_opt10, _, _ = min_variance_optimize(mu_estimate120_data10, varcov10)

In [ ]:
# errori di stima usando i rendimenti attesi reali per portafoglio a 2 asset
realmu_minvar_errs2 = [np.linalg.norm(real_minvar_w_opt2 - week_realmu_minvar_w_opt2, ord=1)/2,
                      np.linalg.norm(real_minvar_w_opt2 - month_realmu_minvar_w_opt2, ord=1)/2,
                      np.linalg.norm(real_minvar_w_opt2 - quarter_realmu_minvar_w_opt2, ord=1)/2]

# errori di stima usando la matrice di varianza covarianza reale per portafoglio a 10 asset
realvar_minvar_errs2 = [np.linalg.norm(real_minvar_w_opt2 - week_realvar_minvar_w_opt2, ord=1)/2,
                       np.linalg.norm(real_minvar_w_opt2 - month_realvar_minvar_w_opt2, ord=1)/2,
                       np.linalg.norm(real_minvar_w_opt2 - quarter_realvar_minvar_w_opt2, ord=1)/2]

# errori di stima usando i rendimenti attesi reali per portafoglio a 10 asset
realmu_minvar_errs10 = [np.linalg.norm(real_minvar_w_opt10 - week_realmu_minvar_w_opt10, ord=1)/10,
                      np.linalg.norm(real_minvar_w_opt10 - month_realmu_minvar_w_opt10, ord=1)/10,
                      np.linalg.norm(real_minvar_w_opt10 - quarter_realmu_minvar_w_opt10, ord=1)/10]

# errori di stima usando la matrice di varianza covarianza reale per portafoglio a 10 asset
realvar_minvar_errs10 = [np.linalg.norm(real_minvar_w_opt10 - week_realvar_minvar_w_opt10, ord=1)/10,
                       np.linalg.norm(real_minvar_w_opt10 - month_realvar_minvar_w_opt10, ord=1)/10,
                       np.linalg.norm(real_minvar_w_opt10 - quarter_realvar_minvar_w_opt10, ord=1)/10]

Facciamo lo stesso con i **portafogli MV**, fissando nuovamente $\lambda = 0.1$ per non far crescere troppo il numero di dati ottenuti.

In [ ]:
## PORTAFOGLIO A 2 Asset
# risultato reale
real_mv_w_opt2, _, _ = risk_aversion_optimize(mu2, varcov2, 0.1)

# stime con rendimenti attesi reali
week_realmu_mv_w_opt2, _, _ = risk_aversion_optimize(mu2, varcov_estimate7_data2, 0.1)
month_realmu_mv_w_opt2, _, _ = risk_aversion_optimize(mu2, varcov_estimate30_data2, 0.1)
quarter_realmu_mv_w_opt2, _, _ = risk_aversion_optimize(mu2, varcov_estimate120_data2, 0.1)

# stime con matrice di varianza-covarianza reale
week_realvar_mv_w_opt2, _, _ = risk_aversion_optimize(mu_estimate7_data2, varcov2, 0.1)
month_realvar_mv_w_opt2, _, _ = risk_aversion_optimize(mu_estimate30_data2, varcov2, 0.1)
quarter_realvar_mv_w_opt2, _, _ = risk_aversion_optimize(mu_estimate120_data2, varcov2, 0.1)


## PORTAFOGLIO A 10 Asset
# risultato reale
real_mv_w_opt10, _, _ = risk_aversion_optimize(mu10, varcov10, 0.1)

# stime con rendimenti attesi reali
week_realmu_mv_w_opt10, _, _ = risk_aversion_optimize(mu10, varcov_estimate7_data10, 0.1)
month_realmu_mv_w_opt10, _, _ = risk_aversion_optimize(mu10, varcov_estimate30_data10, 0.1)
quarter_realmu_mv_w_opt10, _, _ = risk_aversion_optimize(mu10, varcov_estimate120_data10, 0.1)

# stime con matrice di varianza-covarianza reale
week_realvar_mv_w_opt10, _, _ = risk_aversion_optimize(mu_estimate7_data10, varcov10, 0.1)
month_realvar_mv_w_opt10, _, _ = risk_aversion_optimize(mu_estimate30_data10, varcov10, 0.1)
quarter_realvar_mv_w_opt10, _, _ = risk_aversion_optimize(mu_estimate120_data10, varcov10, 0.1)

In [ ]:
# errori di stima usando i rendimenti attesi reali per portafoglio a 2 asset
realmu_mv_errs2 = [np.linalg.norm(real_mv_w_opt2 - week_realmu_mv_w_opt2, ord=1)/2,
                      np.linalg.norm(real_mv_w_opt2 - month_realmu_mv_w_opt2, ord=1)/2,
                      np.linalg.norm(real_mv_w_opt2 - quarter_realmu_mv_w_opt2, ord=1)/2]

# errori di stima usando la matrice di varianza covarianza reale per portafoglio a 10 asset
realvar_mv_errs2 = [np.linalg.norm(real_mv_w_opt2 - week_realvar_mv_w_opt2, ord=1)/2,
                       np.linalg.norm(real_mv_w_opt2 - month_realvar_mv_w_opt2, ord=1)/2,
                       np.linalg.norm(real_mv_w_opt2 - quarter_realvar_mv_w_opt2, ord=1)/2]

# errori di stima usando i rendimenti attesi reali per portafoglio a 10 asset
realmu_mv_errs10 = [np.linalg.norm(real_mv_w_opt10 - week_realmu_mv_w_opt10, ord=1)/10,
                      np.linalg.norm(real_mv_w_opt10 - month_realmu_mv_w_opt10, ord=1)/10,
                      np.linalg.norm(real_mv_w_opt10 - quarter_realmu_mv_w_opt10, ord=1)/10]

# errori di stima usando la matrice di varianza covarianza reale per portafoglio a 10 asset
realvar_mv_errs10 = [np.linalg.norm(real_mv_w_opt10 - week_realvar_mv_w_opt10, ord=1)/10,
                       np.linalg.norm(real_mv_w_opt10 - month_realvar_mv_w_opt10, ord=1)/10,
                       np.linalg.norm(real_mv_w_opt10 - quarter_realvar_mv_w_opt10, ord=1)/10]

Plottiamo infine gli errori al **variare** dell'**orizzonte temporale** su cui sono stati stimati i dati.

In [ ]:
times = ['1 settimana', '1 mese', '4 mesi']
fig = make_subplots(rows=1, cols=2, subplot_titles=['2 asset','10 asset'])

fig.add_trace(go.Scatter(x=times, y=realmu_minvar_errs2, name='Varianza minima - Rendimenti reali', marker={'color':'blue'}, legendgroup='1'), row=1, col=1)
fig.add_trace(go.Scatter(x=times, y=realvar_minvar_errs2, name='Varianza minima - Covarianze reali', marker={'color':'red'}, legendgroup='2'), row=1, col=1)
fig.add_trace(go.Scatter(x=times, y=realmu_mv_errs2, name='Mean variance - Rendimenti reali', marker={'color':'lightgreen'}, legendgroup='3'), row=1, col=1)
fig.add_trace(go.Scatter(x=times, y=realvar_mv_errs2, name='Mean variance - Covarianze reali', marker={'color':'deeppink'}, legendgroup='4'), row=1, col=1)

fig.add_trace(go.Scatter(x=times, y=realmu_minvar_errs10, name='Varianza minima - Rendimenti reali', marker={'color':'blue'}, legendgroup='1', showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=times, y=realvar_minvar_errs10, name='Varianza minima - Covarianze reali', marker={'color':'red'}, legendgroup='2', showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=times, y=realmu_mv_errs10, name='Mean variance - Rendimenti reali', marker={'color':'lightgreen'}, legendgroup='3', showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=times, y=realvar_mv_errs10, name='Mean variance - Covarianze reali', marker={'color':'deeppink'}, legendgroup='4', showlegend=False), row=1, col=2)

fig.update_xaxes(title='Quantità di dati')
fig.update_yaxes(title='Errore', range=[-1,20], col=1)
fig.update_yaxes(range=[-1,70], col=2)
fig.update_layout(height=800)

Riportiamo i seguenti commenti. 
- Il **portafoglio a minima varianza** riesce, in entrambi i casi, a **contenere gli errori** di stima nei **rendimenti attesi** (per ovvi motivi non commette errori nella stima del portafoglio a minima varianza quando viene fornita la matrice di varianza-covarianza reale). 

- Nel caso del **portafoglio MV**, sicuramente la situazione è più delicata:
  -  in entrambi i portafogli, **errori sulla stima dei rendimenti attesi** comportano **grandi errori** nella **stima** dei **portafogli ottimi**. 
  - Per quanto riguarda gli **errori sulla matrice di varianza-covarianza**, essi sembrano avere ben **poco peso** nel portafoglio di **due** soli **asset**, mentre un **peso** decisamente **maggiore** in quello da **dieci** **asset**. \
Una possibile spiegazione è la seguente:  la matrice stimata con i dati di una settimana non è definita positiva, come precedentemente osservato. Tuttavia, questo non può essere l'unico motivo. Infatti, le stime con più dati restituiscono matrici definite positive, ma, nonostante ciò, il problema persiste ancora a i risultati sono peggiori rispetto al caso in cui si considerano errori sui rendimenti attesi. \

- Ancora una volta, come è lecito aspettarsi, ancora una volta, una **maggiore quantità di dati** permette di avere **stime migliori** dei portafogli ottimi.


La regola generale che sembra emergere da questi dati è che, come già osservato in precedenza, il **portafoglio a minima varianza** permette di **contenere** **errori** dovuti alle **stime** dei parametri, mentre quello **MV** è decisamente più **sensibile** a questi ultimi. Nel caso di **portafogli MV**, inoltre, **errori sulla stima dei rendimenti attesi** giocano in ogni caso un ruolo **importante** sulla **correttezza dei risultati**, mentre **errori sulle covarianze** tendono ad amplificare gli errori al **crescere della numerosità degli asset**.

## Confonto tra portafoglio ottimi : l'impatto sulla ricchezza terminale
Nelle precedenti sezioni abbiamo analizzato quale effetto gli errori di stima dei parametri del problema possano avere sulla stima di un portafoglio ottimo.\
 In questa sezione, proveremo invece a **confrontare** i vari approcci di **ottimizzazione** (portafoglio a minima varianza o MV/presenza o meno di vincoli sulle vendite allo scoperto) in base al loro **impatto** sulla **ricchezza terminale.**

Immaginiamo, quindi, di partire da una ricchezza iniziale di €1 e stimiamo la distribuzione di probabilità della ricchezza terminale a 7 giorni e 14 giorni ottenuta utilizzando il portafoglio a minima varianza (sia con vincoli sulle vendite allo scoperto che senza) e il portafoglio MV fissando $\lambda=1$ (sia con vincoli sulle vendite allo scoperto che senza).\

 Concentriamoci sul portafoglio a 10 asset. \

Simuliamo 1000 scenari di rendimenti per i prossimi 14 giorni e utilizziamo solo i portafogli ottimi ottenuti per mezzo delle stime di rendimenti attesi e covarianze su dati dell'ultimo mese e degli ultimi 4 mesi (per evitare i problemi dovuti alla matrice di varianza-covarianza stimata a una settimana, che, come abbiamo osservato, non è definita positiva). 

Calcoliamo infine V@R e CV@R per ogni portafoglio utilizzato.
Per il calcolo di tali misure di rischio, utilizziamo le simulazioni ottenute dai dati reali ( e non dalle stime). 
Tale approccio non è propriamente corretto, ma  ci permette di studiare le conseguenze degli errori di stima sulla richezza terminale, separando dunque gli effetti.  

Osserviamo che gli scenari dei rendimenti dei prossimi 14 giorni vanno simulati utilizzando i dati reali.

In [ ]:
SIZE = 1000
datanext14 = []
for i in range(SIZE):
  datanext14.append(rng.multivariate_normal(mean=mu10, cov=varcov10, size=14).transpose())

In [ ]:
def compute_final_wealths(mu, varcov, realmu, realvarcov):
  minvar_w_opt, _, _ = min_variance_optimize(mu, varcov) # portafoglio a minima varianza
  mv_w_opt, _, _ = risk_aversion_optimize(mu, varcov, 1) # portafoglio mv

  G = -np.identity(10,dtype=np.double) # matrice per vincoli di disuguaglianza
  h = np.zeros(shape=10,dtype=np.double) # vettore per vincoli di disuguaglianza
  A = np.ones(shape=[1,10],dtype=np.double) # matrice per vincoli di uguaglianza
  b = np.array(1,dtype=np.double) # vettore per vincoli di uguaglianza
  minvar_noshort_w_opt = quadprog_solve_qp(varcov, np.zeros(shape=10, dtype=np.double), G, h, A, b) # portafoglio a minima varianza senza vendite allo scoperto
  mv_noshort_w_opt = quadprog_solve_qp(varcov, -mu, G, h, A, b) # portafoglio mv senza vendite allo scoperto

  keys = ['min var', 'mv', 'min var - no short', 'mv - no short']
  portfolios = dict.fromkeys(keys)
  portfolios[keys[0]] = minvar_w_opt
  portfolios[keys[1]] = mv_w_opt
  portfolios[keys[2]] = minvar_noshort_w_opt
  portfolios[keys[3]] = mv_noshort_w_opt

  trajectories = {key:np.ones(shape=datanext14[0].shape[1]+1) for key in keys} # dizionario per salvare la traiettoria di una simulazione per ogni portafoglio

  final_wealth_1w = {key:np.zeros(shape=len(datanext14)) for key in keys} # dizionario per salvare ricchezze a 1 mese per ogni portafoglio (tutte le simulazioni)
  final_wealth_2w = {key:np.zeros(shape=len(datanext14)) for key in keys} # dizionario per salvare ricchezze a 4 mesi per ogni portafoglio (tutte le simulazioni)

  expected_returns = {key: np.dot(realmu, portfolios[key]) for key in keys} # rendimenti attesi reali usando i portafoglio approssimati
  variances = {key: np.dot(portfolios[key], realvarcov.dot(portfolios[key])) for key in keys} # varianze reali usando i portafoglio approssimati

  for portfolio in portfolios.keys():
    for sim in range(len(datanext14)):
      day_value = 1 
      
      for day in range(datanext14[0].shape[1]):
        day_value = day_value * (1+np.dot(portfolios[portfolio],datanext14[sim][:,day]))

        if sim == 0: # se è la prima simulazione, salvo tutta la traiettoria
          trajectories[portfolio][day+1] = day_value

        if day == 7-1 : # se è il 7 giorno, salviamo la ricchezza in quel momento per ogni portafoglio e per ogni simulazione
          final_wealth_1w[portfolio][sim] = day_value
        elif day == 14-1: # idem se è il 14 giorno
          final_wealth_2w[portfolio][sim] = day_value
  
  return trajectories, final_wealth_1w, final_wealth_2w, expected_returns, variances

Iniziamo utilizzando le **stime a 1 mese.**

In [ ]:
trajectories, final_wealth_1w, final_wealth_2w, expected_returns, variances = compute_final_wealths(mu_estimate30_data10, varcov_estimate30_data10, mu10, varcov10)

Plottiamo i risultati ottenuti con la prima simulazione usando i diversi portafogli.

In [ ]:
fig = go.Figure()

for i in trajectories.keys():
  fig.add_trace(go.Scatter(x=np.linspace(0,13,14,dtype=np.int8), y=trajectories[i], name=i))

fig.update_xaxes(type='category', title='giorno')
fig.update_yaxes(type='log', title='€')
fig.show()

Sicuramente i **risultati** di questa simulazione sembrano ben **promettenti** per il **portafoglio MV**.
 Indaghiamo però meglio i risultati.

Calcoliamo adesso V@R e CV@R della ricchezza a 7 e 14 giorni per ogni portafoglio e plottiamola.

In [ ]:
def compute_var(data):
  var = {key : np.quantile(data[key], 0.05) for key in data.keys()}
  return var

def compute_cvar(data):
  var = compute_var(data)
  cvar = {key : np.mean(data[key][data[key] <= var[key]]) for key in data.keys()}
  return cvar

In [ ]:
print('V@R della ricchezza a 1 settimana',compute_var(final_wealth_1w))
print('CV@R della ricchezza a 1 settimana',compute_cvar(final_wealth_1w))
print('V@R della ricchezza a 2 settimane',compute_var(final_wealth_2w))
print('CV@R della ricchezza a 2 settimane',compute_cvar(final_wealth_2w))

V@R della ricchezza a 1 settimana {'min var': 15.055843757527567, 'mv': 50935810.10125781, 'min var - no short': 0.6180525388253533, 'mv - no short': 0.7550425993351753}
CV@R della ricchezza a 1 settimana {'min var': 13.484989020297327, 'mv': -45785346.49691123, 'min var - no short': 0.4616375212499025, 'mv - no short': 0.5658785855788008}
V@R della ricchezza a 2 settimane {'min var': 294.30764567280056, 'mv': 4161132288307584.0, 'min var - no short': 0.7996750689700223, 'mv - no short': 1.2572814182598568}
CV@R della ricchezza a 2 settimane {'min var': 241.41817316658324, 'mv': -7.306661474637347e+16, 'min var - no short': 0.5471854379194144, 'mv - no short': 0.8330797326413986}


Notiamo che il portafoglio MV ha CV@R negativo. \
Avendo utilizzato una scala logaritmica per i plot, esso non verrà visualizzato.

In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=['Ricchezza terminale dopo 7 giorni', 'Ricchezza terminale dopo 14 giorni'], specs=[[{}, {}], [{'colspan':2}, None]], vertical_spacing=0.08, row_heights=[0.6,0.4])

fig.add_trace(go.Bar(x=list(final_wealth_1w.keys()), y=list(compute_var(final_wealth_1w).values()), name='V@R', legendgroup=1, marker=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Bar(x=list(final_wealth_1w.keys()), y=list(compute_cvar(final_wealth_1w).values()), name='CV@R', legendgroup=2, marker=dict(color='red')), row=1, col=1)

fig.add_trace(go.Bar(x=list(final_wealth_1w.keys()), y=list(compute_var(final_wealth_2w).values()), name='V@R', legendgroup=1, showlegend=False, marker=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Bar(x=list(final_wealth_1w.keys()), y=list(compute_cvar(final_wealth_2w).values()), name='CV@R', legendgroup=2, showlegend=False, marker=dict(color='red')), row=1, col=2)

fig.add_trace(go.Bar(x=list(expected_returns.keys()), y=list(expected_returns.values()), name='Rendimento atteso', legendgroup=3), row=2, col=1)
fig.add_trace(go.Bar(x=list(variances.keys()), y=list(variances.values()), name='Varianza', legendgroup=4), row=2, col=1)

fig.update_yaxes(type='log')
fig.update_layout(height=1000)

Come possiamo osservare dai grafici, i **risultati** sono molto **simili** per entrambi gli orizzonti temporali.
- Affidandoci al **V@R**, la scelta migliore è il portafoglio MV con vendite allo scoperto. Purtroppo quest'ultimo è anche il portafoglio che ha il peggior CV@R. La motivazione è sicuramente, come mostra l'ultimo grafico, l'elevatissima **varianza** del **rendimento** del **portafoglio**. 
-Affidandoci al **CV@R**, invece, il portafoglio migliore è quello a minima varianza con vendite allo scoperto, che può essere in generale considerato come l'**alternativa migliore** in entrambi i casi.

Facciamo adesso gli stessi ragionamenti usando le stime a 4 mesi.

In [ ]:
trajectories, final_wealth_1w, final_wealth_2w, expected_returns, variances = compute_final_wealths(mu_estimate120_data10, varcov_estimate120_data10, mu10, varcov10)

In [ ]:
fig = go.Figure()

for i in trajectories.keys():
  fig.add_trace(go.Scatter(x=np.linspace(0,13,14,dtype=np.int8), y=trajectories[i], name=i))

fig.update_xaxes(type='category', title='giorno')
fig.update_yaxes(type='log', title='€')
fig.show()

Ancora una volta, il **portafoglio MV** sembra essere il **più** **promettente**, anche in questa simulazione.

In [ ]:
print('V@R della ricchezza a 1 settimana',compute_var(final_wealth_1w))
print('CV@R della ricchezza a 1 settimana',compute_cvar(final_wealth_1w))
print('V@R della ricchezza a 2 settimane',compute_var(final_wealth_2w))
print('CV@R della ricchezza a 2 settimane',compute_cvar(final_wealth_2w))

V@R della ricchezza a 1 settimana {'min var': 13.839771426432257, 'mv': 6215078.900043517, 'min var - no short': 0.7279501554740281, 'mv - no short': 0.8333700295841322}
CV@R della ricchezza a 1 settimana {'min var': 12.598351564284894, 'mv': 3897055.840675428, 'min var - no short': 0.5559927680855414, 'mv - no short': 0.6167739530203606}
V@R della ricchezza a 2 settimane {'min var': 244.9481182793656, 'mv': 84184870244356.05, 'min var - no short': 1.0772536397478332, 'mv - no short': 1.4779619720766204}
CV@R della ricchezza a 2 settimane {'min var': 209.43080812698867, 'mv': 47171902834155.94, 'min var - no short': 0.767617520759585, 'mv - no short': 1.029377343163218}


In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=['Ricchezza terminale dopo 7 giorni', 'Ricchezza terminale dopo 14 giorni'], specs=[[{}, {}], [{'colspan':2}, None]], vertical_spacing=0.08, row_heights=[0.6,0.4])

fig.add_trace(go.Bar(x=list(final_wealth_1w.keys()), y=list(compute_var(final_wealth_1w).values()), name='V@R', legendgroup=1, marker=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Bar(x=list(final_wealth_1w.keys()), y=list(compute_cvar(final_wealth_1w).values()), name='CV@R', legendgroup=2, marker=dict(color='red')), row=1, col=1)

fig.add_trace(go.Bar(x=list(final_wealth_1w.keys()), y=list(compute_var(final_wealth_2w).values()), name='V@R', legendgroup=1, showlegend=False, marker=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Bar(x=list(final_wealth_1w.keys()), y=list(compute_cvar(final_wealth_2w).values()), name='CV@R', legendgroup=2, showlegend=False, marker=dict(color='red')), row=1, col=2)

fig.add_trace(go.Bar(x=list(expected_returns.keys()), y=list(expected_returns.values()), name='Rendimento atteso', legendgroup=3), row=2, col=1)
fig.add_trace(go.Bar(x=list(variances.keys()), y=list(variances.values()), name='Varianza', legendgroup=4), row=2, col=1)

fig.update_yaxes(type='log')
fig.update_layout(height=1000)

In questo caso, il **portafoglio MV** sembra davvero essere la **scelta migliore**, sia dal punto di vista del **V@R** che da quello del **CV@R**.

 Possiamo notare anche  che i portafogli che **non** prevedono **vendite allo scoperto**, nonostante fossero utili a ridurre gli errori di stima, sono i meno performanti dal punto di vista della ricchezza terminale.
 
E' interessante notare che, in questo caso, come in quello precedente (con dati stimati a 30 giorni), il **portafoglio a minima varianza** generi comunque dei **risultati molto buoni** sulla ricchezza terminale.

 Infine, nonostante i **vincoli** sulle **vendite** allo **scoperto** abbiano l'effetto di ridurre gli errori di stima, hanno anche quello di **aumentare** la **varianza** del **portafoglio a minima varianza**. La spiegazione può essere la seguente: imponendo un nuovo vincolo, stiamo riducendo la regione di ammissibilità del problema.